In [1]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화

import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [2]:
# 검색어
keyword1 = input("1.크롤링할 키워드를 입력하세요: ")

'''
#검색할 키워드 정리
10대 여자 어린이날 선물
10대 남자 어린이날 선물

20,30대 엄마 어버이날 선물
20,30대 아빠 어버이날 선물
40,50대 엄마 어버이날 선물
40,50대 아빠 어버이날 선물
60대 엄마 어버이날 선물
60대 아빠 어버이날 선물

20,30대 여자 선생님 스승의날 선물
20,30대  남자 선생님 스승의날 선물
40,50대 여자 선생님 스승의날 선물
40,50대 남자 선생님 스승의날 선물
60대 여자 선생님 스승의날 선물
60대 남자 선생님 스승의날 선물

10대 여자 생일 선물
10대 남자 생일 선물
20,30대 여자 생일 선물
20,30대 남자 생일 선물
40,50대 여자 생일 선물
40,50대 남자 생일 선물
60대 여자 생일 선물
60대 남자 생일 선물

20,30대 여자 결혼기념일 선물
20,30대 남자 결혼기념일 선물
40,50대 여자 결혼기념일 선물
40,50대 남자 결혼기념일 선물
60대 여자 결혼기념일 선물
60대 남자 결혼기념일 선물
'''

1.크롤링할 키워드를 입력하세요: 10대 여자 어린이날 선물


'\n#검색할 키워드 정리\n10대 여자 어린이날 선물\n10대 남자 어린이날 선물\n\n20,30대 엄마 어버이날 선물\n20,30대 아빠 어버이날 선물\n40,50대 엄마 어버이날 선물\n40,50대 아빠 어버이날 선물\n60대 엄마 어버이날 선물\n60대 아빠 어버이날 선물\n\n20,30대 여자 선생님 스승의날 선물\n20,30대  남자 선생님 스승의날 선물\n40,50대 여자 선생님 스승의날 선물\n40,50대 남자 선생님 스승의날 선물\n60대 여자 선생님 스승의날 선물\n60대 남자 선생님 스승의날 선물\n\n10대 여자 생일 선물\n10대 남자 생일 선물\n20,30대 여자 생일 선물\n20,30대 남자 생일 선물\n40,50대 여자 생일 선물\n40,50대 남자 생일 선물\n60대 여자 생일 선물\n60대 남자 생일 선물\n\n20,30대 여자 결혼기념일 선물\n20,30대 남자 결혼기념일 선물\n40,50대 여자 결혼기념일 선물\n40,50대 남자 결혼기념일 선물\n60대 여자 결혼기념일 선물\n60대 남자 결혼기념일 선물\n'

수집할 url 크롤링

In [3]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(r"D:\정보현\chromedriver.exe")

# 사이트 주소
driver.get("http://www.naver.com")
time.sleep(1)

# 검색창에 '검색어' 검색
element = driver.find_element_by_id("query") #f12눌러서 확인
element.send_keys(keyword1)
element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click()

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click()
time.sleep(1)

# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 10
i = 0
while i < n:
    scroll_down(driver)
    i = i+1
    
# 블로그 글 url들 수집
url_list = []

# URL_raw 크롤링 시작
article_raw = driver.find_elements_by_css_selector(".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

time.sleep(1)
    
print("url개수: ", len(url_list))

df = pd.DataFrame({'url':url_list})
df

# url_list 저장
df.to_excel("naver_blog_url.xlsx")

url개수:  330


블로그 내용 크롤링

In [4]:
import sys
import os

import pandas as pd
import numpy as np

In [17]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("naver_blog_url.xlsx")

num_list = len(url_load)

print(num_list)
#url_load

330


In [18]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = num_list    # 수집할 글 갯수

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load["url"][i]
    driver = webdriver.Chrome(r"D:\정보현\chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        
        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)
    
        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['content'] = content_str
        

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        #time.sleep(1)


        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간 저장 (없어도됨)
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')   # 한글은 오류날 수 있어서 encoding 추가
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
#print(dict)

C:\Users\minno\AppData\Local\Temp\ipykernel_22468\303234191.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, number)):


  0%|          | 0/330 [00:00<?, ?it/s]

수집한 글 갯수:  285


In [19]:
# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
#result_df

# 엑셀 저장
result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')

액셀 데이터 텍스트 파일로 변환하기

In [20]:
#엑셀 파일을 읽어오는 소스 코드는 
tf=pd.read_excel('naver_blog_content.xlsx', usecols = ['content'])
tf.to_csv("content.txt", index=True)

형태소 분석

In [67]:
#파일 오픈
of = open("content.txt","r",encoding="utf-8")
data=of.read()
data

',content\n0," 이제 곧 가정의달 5월이네요!\n지갑에 구멍은 나지만 마음은\n행복한 달 맞죵?ㅋㅋㅋ\n우래기들 어린이날선물과 어른이를 위한\n폴햄 여름신상할인정보 가져왔어요:)  아이들과 마트가면 그냥 지나치지못하는\n장난감코너🚂⚽️🪅🧸ㅋㅋㅋㅋㅋ\n어린이날 다가오니 뭔가 더 가득차있는\n느낌이더라구요?^^ ✔️ 10대캐릭터 대전 최대50%할인!\n✔️ 완구6만원/ 전자게임10만원이상 \n카드 구매시 1만원 롯데상품권 증정\n(~5월8일까지)\n완전 꿀행사아닙니까?ㅋㅋㅋ\n우래기들 직접 가서 사는거 더 기분나고\n좋아하자나용? 이럴때 똭!!! 와ㅋㅋㅋ꽤 쏠쏠한 할인이라\n지갑가벼워지는 5월에 신나는 소식입니다!\n띠우가 사랑하는 카봇까지ㅋㅋㅋ 뽀통령님도 당연시 있으시구여ㅋㅋㅋ 여자친구들의 최애 시크릿쥬쥬님ㅋㅋ\n어른이봐도 이뿜ㅋㅋㅋ 겨울이면 엘사로 변신하는 공주님덜\n좋아할 코너ㅋㅋㅋ30프로할인품목\n야무지게 챙기시구여!!! 남자친구들이 사랑하는 레고할인까지!\n죠으다죠으다~~~♡♡♡ 10대캐릭터 외에도 어린이날을 맞이해서\n할인하는 품목들이 또 있으니\n잘 둘러보시고 득템해서 우래기들\n행복한 어린이날 만들어주시길바래용ㅋㅋ  롯마갔다가 폴햄행사발견!!!\n예전엔 몰랐는데 폴햄도 가성비좋고\n깔끔한옷들이 많더라구요?\n요럴때 겟해야쥬ㅋㅋㅋㅋㅋ\n✔️ 행사기간 ~5월10일까지  기본티부터 조거팬츠, 바람막이 등\n착한가격으로 여름신상을 겟할 수 있는 기회!\n놓치지마시길용ㅋㅋㅋㅋㅋ 🌟 롯데마트 통영점 🌟\n매일 10:00 ~ 23:00\n휴무일 매월2,4주차 일요일  #롯데마트 #롯데마트토이저러스\n#토이저러스세일 #어린이날세일\n#롯데마트어린이날행사 #롯마세일정보\n#폴햄행사 #폴햄세일 #폴햄1+1행사\n#폴햄여름신상품 #통영롯데마트세일\n#어린이날선물 #어린이날할인정보"\n1,"다가오는 어린이날, 조카선물 고민하는 삼촌이모고모들에게 추천하는\n10대 여학생 지갑추천 \'리사벨\' 지갑\n어린이날 선물 추천, 10대 여자 학생 지갑 추천 \'리사벨\

https://thinkwarelab.wordpress.com/2016/08/30/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EC%9C%BC%EB%A1%9C-%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C-%EA%B7%B8%EB%A6%AC%EA%B8%B0/

In [68]:
#!pip install konlpy
from konlpy.tag import Okt
okt=Okt()  #Okt 객체를 생성

nouns = okt.nouns(data)

In [69]:
#명사 추출
nouns

['이제',
 '곧',
 '가정',
 '달',
 '지갑',
 '구멍',
 '나',
 '마음',
 '달',
 '맞죵',
 '기',
 '어린이날',
 '선물',
 '어른',
 '위',
 '폴햄',
 '여름',
 '신상',
 '할인',
 '정보',
 '아이',
 '마트',
 '가면',
 '그냥',
 '장난감',
 '코너',
 '어린이날',
 '뭔가',
 '더',
 '가득',
 '캐릭터',
 '대전',
 '최대',
 '할인',
 '완구',
 '전자',
 '게임',
 '카드',
 '구매',
 '시',
 '롯데',
 '상품권',
 '증정',
 '완전',
 '꿀',
 '행사',
 '기',
 '직접',
 '더',
 '분나',
 '때',
 '똭',
 '꽤',
 '할인',
 '지갑',
 '소식',
 '띠우',
 '사랑',
 '카봇',
 '뽀',
 '통령',
 '시',
 '여자친구',
 '최애',
 '시크릿',
 '쥬쥬',
 '어른',
 '겨울',
 '이면',
 '엘사',
 '변신',
 '공주',
 '덜',
 '코너',
 '프로',
 '할인',
 '품목',
 '남자친구',
 '사랑',
 '레고',
 '할인',
 '죠으다죠으',
 '캐릭터',
 '외',
 '어린이날',
 '맞이',
 '할인',
 '품목',
 '또',
 '득템',
 '기',
 '어린이날',
 '바',
 '용',
 '롯마',
 '폴햄',
 '행사',
 '발견',
 '예전',
 '폴햄',
 '가성',
 '비',
 '옷',
 '요럴때',
 '겟해야쥬',
 '행사',
 '기간',
 '티',
 '조거',
 '팬츠',
 '바람막이',
 '등',
 '가격',
 '여름',
 '신상',
 '겟할',
 '수',
 '기회',
 '용',
 '롯데',
 '마트',
 '통영',
 '점',
 '매일',
 '휴무',
 '일',
 '매월',
 '주차',
 '일요일',
 '행사',
 '어린이날',
 '조카',
 '선물',
 '고민',
 '삼촌',
 '고모',
 '추천',
 '여학생',
 '지갑',
 '추천',
 '리사',
 '벨',
 '지갑',
 

In [70]:
#불용어 처리
stopword = ['선물','어린이날','날','여자','아이','어린이','추천', '엄마', '나', '구매', '때', '수', '것', '더', '이', '저', '가격', '말', 
            '그', '생각', '사용', '제', '제품', '우리', '친구', '거', '조카', '요즘', '등', '위', '요', '집', '하나', '준비', '전', '가지', 
           '곳', '중', '때문', '사진', '할인', '브랜드', '남자', '내', '정말', '번', '모습', '행사', '영화', '사람', '어버이날', '부모님',
            '피부', '박스', '건강', '분', '느낌', '마음', '기능', '안', '고민', '이번', '아빠', '갱년기', '여성', '도움', '해', '성분', 
            '섭취', '관리', '달','선택', '바로', '건', '걸', '포장', '맛', '도', '포', '몸', '디자인', '선생님', '스승의날', '스승', '골드',
           '다니엘', '웰링턴', '향', '착용', '어린이집', '세트', '주문', '개', '생일', '로', '시간', '또', '후', '기분', '캡슐', '카네이션',
            '스트랩', '남자친구', '부담', '오늘', '위해', '감사', '가장', '가정', '리본', '정도', '게', '학생', '학교', '기억', '남성', 
            '수분', '감', '케어', '컬러', '코드', '데', '다이얼', '여자친구', '일리', '가죽', '좀', '남편', '조금', '환', '물', '프로바이오틱스', 
            '나이', '아버지', '저희', '효과', '그냥', '생신', '사실', '부분', '경우', '면도', '장', '에스트로겐', '메노락토', '결혼기념일', 
            '줄', '원료', '꽃집', '꽃다발', '어머니', '문의', '부모', '제작', '졸업', '플라워', '사이즈', '일', '결혼', '아내', '현재', 
            '지금', '시작', '가족', '부부', '김자', '보고', '알', '왜', '살', '이제', '아들', '하늘정원', '인스타', '사랑', '화', '맘', '패밀리', '꼭', '진짜', '운동', '블로그', '카카오스토리', '오후', '다시', '자신', '조선', '마이', '명', '못', '의', '한번', '너','공부', '딸', '앞', '이야기', '인생', '네', '도티','방송','를','추억','처음', '다른', '책', '촬영', '한국', '듯', '다음', '차', '대한','눈','하루','맛집','여기','속','이후','모두','은','사회','활동']
ko = [i for i in nouns if i not in stopword]
ko

['곧',
 '지갑',
 '구멍',
 '맞죵',
 '기',
 '어른',
 '폴햄',
 '여름',
 '신상',
 '정보',
 '마트',
 '가면',
 '장난감',
 '코너',
 '뭔가',
 '가득',
 '캐릭터',
 '대전',
 '최대',
 '완구',
 '전자',
 '게임',
 '카드',
 '시',
 '롯데',
 '상품권',
 '증정',
 '완전',
 '꿀',
 '기',
 '직접',
 '분나',
 '똭',
 '꽤',
 '지갑',
 '소식',
 '띠우',
 '카봇',
 '뽀',
 '통령',
 '시',
 '최애',
 '시크릿',
 '쥬쥬',
 '어른',
 '겨울',
 '이면',
 '엘사',
 '변신',
 '공주',
 '덜',
 '코너',
 '프로',
 '품목',
 '레고',
 '죠으다죠으',
 '캐릭터',
 '외',
 '맞이',
 '품목',
 '득템',
 '기',
 '바',
 '용',
 '롯마',
 '폴햄',
 '발견',
 '예전',
 '폴햄',
 '가성',
 '비',
 '옷',
 '요럴때',
 '겟해야쥬',
 '기간',
 '티',
 '조거',
 '팬츠',
 '바람막이',
 '여름',
 '신상',
 '겟할',
 '기회',
 '용',
 '롯데',
 '마트',
 '통영',
 '점',
 '매일',
 '휴무',
 '매월',
 '주차',
 '일요일',
 '삼촌',
 '고모',
 '여학생',
 '지갑',
 '리사',
 '벨',
 '지갑',
 '지갑',
 '리사',
 '벨',
 '여학생',
 '지갑',
 '심플',
 '여학생',
 '지갑',
 '실물',
 '영접',
 '더욱',
 '여학생',
 '초등학생',
 '지갑',
 '리사',
 '벨',
 '구성',
 '질의',
 '여학생',
 '초등학생',
 '지갑',
 '리사',
 '벨',
 '동전',
 '공간',
 '지퍼',
 '라인',
 '가성',
 '비',
 '여학생',
 '초등학생',
 '지갑',
 '리사',
 '벨',
 '심플',
 '구성',
 '수납',
 '공간',
 '여학생',
 '초등학생',
 '지갑',
 '리사',
 '

In [71]:
#빈도수 체크
from collections import Counter
count = Counter(ko)
#count

#상위 3 출력
count.most_common(3)

[('장난감', 255), ('지갑', 191), ('보드', 153)]

https://dalulu.tistory.com/108
http://bigdata.dongguk.ac.kr/lectures/DB/_book/python%EC%97%90%EC%84%9C-mysql%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%98-%EC%A0%91%EA%B7%BC.html

In [72]:
#딕셔너리 생성 후 키값과 value값 분리
tag_count=[]
tags=[]

for n,c in count.most_common(3):
    dics={'tag': n, 'count':c}
    tag_count.append(dics)
    tags.append(dics['tag'])

https://hyun-am-coding.tistory.com/entry/%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0-DB%EC%97%90-%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0
https://codechacha.com/ko/python-mysql-select-order/

In [73]:
!pip install mysqlclient
!pip install pymysql

In [74]:
#DB 셍성, 테이블 생성, 데이터 추가(처음 코드를 실행할 때 사용)

import MySQLdb

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
    # charset="utf-8"
)
# 커서 생성하기
cursor = conn.cursor()

# 실행할 때마다 다른값이 나오지 않게 테이블을 제거해두기
cursor.execute("DROP TABLE IF EXISTS present")

# 테이블 생성하기
cursor.execute("CREATE TABLE present (`rank` int, situation text, gender text, age text, present text)")

# 데이터 저장하기
situation = "Children"    #Children, Parents, Teacher, Birthday, Wedding
gender = "woman"    #man, woman
age = "ten"    #ten, twenty, forty, sixty 
i=1

for tag in tag_count:
    cursor.execute(f"INSERT INTO present VALUES({i},\"{situation}\", \"{gender}\", \"{age}\", \"{tag['tag']}\")")
    i+=1

# 커밋하기
conn.commit()

# 연결종료하기
conn.close()

https://codechacha.com/ko/python-mysql-insert-update-delete/

In [80]:
#데이터 추가(DB 및 테이블 생성 후 데이터만 추가하기)

import MySQLdb

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
)

# 커서 생성하기
cursor = conn.cursor()


# 데이터 저장하기
situation = "Wedding"
gender = "man"
age = "sixty"
i=1

for tag in tag_count:
    cursor.execute(f"INSERT INTO present VALUES({i},\"{situation}\", \"{gender}\", \"{age}\", \"{tag['tag']}\")")
    i+=1


# 커밋하기
conn.commit()

In [81]:
import MySQLdb

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
)

# 테이블 불러옴
with conn:
    with conn.cursor() as cursor:
        #cursor.execute("SELECT * FROM recommend")
        cursor.execute("SELECT * FROM recommend WHERE situation LIKE %s AND age LIKE %s", ['Teacher', 'sixty'])
        result=cursor.fetchall()
        print(result)


((1, 'Teacher', 'woman', 'sixty', '꽃'), (2, 'Teacher', 'woman', 'sixty', '콜라겐'), (3, 'Teacher', 'woman', 'sixty', '크림'), (1, 'Teacher', 'man', 'sixty', '꽃'), (2, 'Teacher', 'man', 'sixty', '과일'), (3, 'Teacher', 'man', 'sixty', '편지'))


In [9]:
#db 출력
import MySQLdb
conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
    # charset="utf-8"
)
# 커서 생성하기
cursor = conn.cursor()
cursor.execute("SELECT * FROM recommend")
result = cursor.fetchall()
a = pd.DataFrame(result)
print(a)

    0         1      2      3    4
0   1  Children  woman    ten  장난감
1   2  Children  woman    ten   지갑
2   3  Children  woman    ten   보드
3   1  Children    man    ten  장난감
4   2  Children    man    ten   가방
.. ..       ...    ...    ...  ...
79  2   Wedding  woman  sixty  귀걸이
80  3   Wedding  woman  sixty   반지
81  1   Wedding    man  sixty    꽃
82  2   Wedding    man  sixty    돈
83  3   Wedding    man  sixty  케이크

[84 rows x 5 columns]


In [2]:
!pip install flask
from flask import Flask
app = Flask(__name__)

@app.route('/' ,methods=['POST','GET'])
def hello_world():
    return 'Bye World!'

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [ ]:
import pymysql
import pandas as pd
from flask import Flask
import MySQLdb

def db_connector():
    conn = MySQLdb.connect(user="crawl_usr", passwd="Test001", host="localhost", db="crawl_data", charset="utf-8")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM recommend")
    result = cursor.fetchall()
    db.close()
    return str(result)

app = Flask(__name__)
@app.route('/')
def index():
    a = db_connector()
    return a
if __name__ == "__main__":              
    app.run(debug=True)